In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE

#  Load the dataset
df = pd.read_csv('Train_Test_IoT_GPS_Tracker.csv')


# Convert latitude and longitude to numeric 
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Handling missing values (mean for numerical, mode for categorical)
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values for numerical columns
df[['latitude', 'longitude']] = num_imputer.fit_transform(df[['latitude', 'longitude']])

# Impute missing values for the categorical column 'type'
df[['type']] = cat_imputer.fit_transform(df[['type']])

#  Handling Outliers using Isolation Forest (for numerical columns)
iso = IsolationForest(contamination=0.01)  # Tune contamination rate if needed
outliers = iso.fit_predict(df[['latitude', 'longitude']])

# Remove rows identified as outliers (-1 label from Isolation Forest)
df = df[outliers != -1]

# One-Hot Encoding for categorical column 'type'
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap
type_encoded = pd.DataFrame(encoder.fit_transform(df[['type']]), 
                            columns=encoder.get_feature_names_out(['type']))

# Concatenate the encoded columns back to the main dataframe and drop the original 'type' column
df = pd.concat([df, type_encoded], axis=1)
df.drop('type', axis=1, inplace=True)

# Standardize numerical columns ('latitude' and 'longitude')
scaler = StandardScaler()
df[['latitude', 'longitude']] = scaler.fit_transform(df[['latitude', 'longitude']])

# Drop non-numeric columns like 'date' and 'time'
df.drop(['date', 'time'], axis=1, inplace=True)

#  Handle Missing Values in All Columns (features X)
# Impute any remaining missing values in X (features) before SMOTE
imputer_all = SimpleImputer(strategy='mean')  
df_imputed = pd.DataFrame(imputer_all.fit_transform(df), columns=df.columns)

#  Convert 'label' column to integer if needed
df_imputed['label'] = df_imputed['label'].astype(int)  # Ensure the label is treated as a classification target

#  Handling Imbalanced Data using SMOTE
smote = SMOTE(sampling_strategy='auto')  # 'auto' balances all classes
X = df_imputed.drop(['label'], axis=1)  # All features except the label
y = df_imputed['label']  # Target labels

# Apply SMOTE to balance the classes
X_smote, y_smote = smote.fit_resample(X, y)

# Combine back the balanced dataset
df_balanced = pd.concat([X_smote, y_smote], axis=1)

# Step 9: Save the preprocessed dataset
df_balanced.to_csv('Preprocessed_GPS_Tracker_Dataset.csv', index=False)

print("Preprocessing complete. Dataset saved as 'Preprocessed_GPS_Tracker_Dataset.csv'.")


C:\Users\krmic\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Preprocessing complete. Dataset saved as 'Preprocessed_GPS_Tracker_Dataset.csv'.
